In [1]:
import matplotlib.pyplot as plt
import random
import numpy as np
import pandas as pd
from matplotlib.ticker import LogFormatter 
from scipy.optimize import curve_fit
from pathlib import Path
import matplotlib.colors
import regex as re
from matplotlib import ticker, cm

Reference genome: ftp://ftp.ncbi.nlm.nih.gov/genomes/genbank/bacteria/Escherichia_coli/all_assembly_versions/GCA_000005845.2_ASM584v2/

In [2]:
#!wget ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/005/845/GCF_000005845.2_ASM584v2/GCF_000005845.2_ASM584v2_genomic.fna.gz

In [3]:
#!gunzip *.gz

In [4]:
#! echo "y" | conda install -c bioconda emboss 

In [5]:
#Считывание 
from Bio import SeqIO
record = SeqIO.read("GCF_000005845.2_ASM584v2_genomic.fna", "fasta")
print("%s %i" % (record.id, len(record)))

NC_000913.3 4641652


In [6]:
record.seq

Seq('AGCTTTTCATTCTGACTGCAACGGGCAATATGTCTCTGTGTGGATTAAAAAAAG...TTC')

In [7]:
4000000/4641652 #надо обработать пересечения аккуратно 

0.8617621484764476

In [8]:
genome_string = record.seq
n = 1000
l = 4000
#случайно выберем 1000 индексов  и будем брать по ним 
k = len(genome_string)//n
indexes = np.random.randint(len(genome_string)//n-4000, size=n)  

seq_1000 = [] 
for i in range(len(indexes)):
    start = i*k+indexes[i]
    seq_1000.append(genome_string[start:start+l])

In [39]:
nucleotides = ['A', 'T', 'G', 'C']

def freq_k1(genome_array):
    l = len(genome_array[0])*len(genome_array)  
    frequencies = { 'A' : 0, 'T' : 0,'G' : 0,'C' : 0}
    for genome in genome_array:
        for nucleotide in nucleotides:
            frequencies[nucleotide] += genome.count(nucleotide) / l 
    return frequencies

#Возможно, не самыц красивый способ получить частоты, и можно рекурсивно
#но главное, что работает

def freq_k2(genome_array):      
    frequencies = {}    
    for first in nucleotides:
        for second in nucleotides:     
            frequencies.update({first+second : 0 })   
    num_dinucleotides = (len(genome_array[0]) - 1.0)   
    for genome in genome_array:
        
        for dinucleotide in frequencies.keys(): 
            frequencies[dinucleotide] = 4*len(re.findall(dinucleotide, str(genome), overlapped=True)) / num_dinucleotides
    
    return frequencies 

def freq_k3(genome_array):      
    frequencies = {}    
    for first in nucleotides:
        for second in nucleotides:
            for third in nucleotides:
                frequencies.update({first+second + third : 0 })   
    num_trinucleotides = (len(genome_array[0]) - 2.0)   
    
    for genome in genome_array:       
        for trinucleotide in frequencies.keys(): 
            frequencies[trinucleotide] = 16*len(re.findall(trinucleotide, str(genome), overlapped=True)) / num_trinucleotides
    
    return frequencies 

def freq_k4(genome_array): 
    frequencies = {}    
    for first in nucleotides:
        for second in nucleotides:
            for third in nucleotides:
                for fourth in nucleotides:
                    frequencies.update({first+second + third + fourth : 0 })
    num_quadnucleotides = (len(genome_array[0]) - 3.0)   
    
    for genome in genome_array:
        for quadnucleotide in frequencies.keys(): 
            frequencies[quadnucleotide] = 64*len(re.findall(quadnucleotide, str(genome), overlapped=True)) / num_quadnucleotides
    
    return frequencies 

Ниже просто тесты для проверки, что сумма частот равна единице 

In [40]:
freq_k1(seq_1000)

{'A': 0.24641099999999988,
 'T': 0.24600625000000004,
 'G': 0.2535025,
 'C': 0.2540802500000002}

In [44]:
def log_bern(genome_array, test):
    #Правдоподобие для Бернулевской модели
    frequencies = freq_k1(genome_array)
    total_probability = 0
    for nucleotide in nucleotides:
        total_probability += test.count(nucleotide) * np.log(frequencies[nucleotide])
                
    return total_probability

In [45]:
def log_markov(genome_array, test, order=1):
    #Правдободобие для Марковских моделей порядка 1, 2 или 3
        
    nucleotide_frequencies = freq_k1(genome_array)
    
    if (order==1):
        frequencies = freq_k2(genome_array)
    elif (order==2):
        frequencies = freq_k3(genome_array)
    else:
        frequencies = freq_k4(genome_array)        
    counts = {}
    for seq in frequencies.keys():
        counts[seq] = len(re.findall(seq, test, overlapped=True))
                              
    total_probability = 0
    
    for nucleotide in test[0:order]:
        total_probability += np.log(nucleotide_frequencies[nucleotide])
        
    transitions = frequencies
    for seq in transitions:
        total_probability += counts[seq] * np.log(transitions[seq])
        
    return total_probability

In [46]:
record = SeqIO.read("GCF_000005845.2_ASM584v2_genomic.fna", "fasta")

Бернулевская модель: 

In [47]:
n = 1000 #Размер обучающей выборки
logprob = log_bern(seq_1000, str(record.seq))
m = 4  #число параметров 
AIC = 2*m - 2*logprob
BIC = np.log(n)*m - 2*logprob
print("AIC = ", round(AIC, 2))
print("BIC = ", round(BIC, 2)) 
 

AIC =  12868237.39
BIC =  12868257.02


Модели Маркова: 

In [48]:
n = 1000 #Размер обучающей выборки

print("I order: ")
logprob = log_markov(seq_1000, str(record.seq), 1)
m = 16  #число параметров 
AIC = 2*m - 2*logprob
BIC = np.log(n)*m - 2*logprob
print("AIC = ", round(AIC, 2))
print("BIC = ", round(BIC, 2)) 
 
print("II order: ")
logprob = log_markov(seq_1000, str(record.seq), 2)
m = 4**3  #число параметров 
AIC = 2*m - 2*logprob
BIC = np.log(n)*m - 2*logprob
print("AIC = ", round(AIC, 2))
print("BIC = ", round(BIC, 2)) 

print("III order: ")
logprob = log_markov(seq_1000, str(record.seq), 3)
m = 4**4  #число параметров 
AIC = 2*m - 2*logprob
BIC = np.log(n)*m - 2*logprob
print("AIC = ", round(AIC, 2))
print("BIC = ", round(BIC, 2)) 

I order: 
AIC =  12810655.76
BIC =  12810734.29
II order: 
AIC =  12683943.08
BIC =  12684257.17
III order: 
AIC =  12739321.35
BIC =  12740577.73


Самые маленький значений критериев BIC и AIC получились для модели Маркова второго порядка.
Можно предположить, что эта модель наилучшим образом описывается структуру днк.